In [2]:
from scapy.all import rdpcap, TCP, IP
from pathlib import Path
import pandas as pd


In [3]:
# =============================
# PATH PCAP (LANGSUNG)
# =============================
PCAP_NO_IPS = Path(r"D:\BOOSTIFY-IOTML\snort\pengujian\goldeneye\serangan_A.pcap")
PCAP_IPS    = Path(r"D:\BOOSTIFY-IOTML\snort\pengujian_model\ae\goldeneye\serangan_pengujian.pcap")

# =============================
# FUNGSI ANALISIS QoS (VALID)
# =============================
def analyze_qos(pcap_path, label):
    packets = rdpcap(str(pcap_path))
    rows = []

    for pkt in packets:
        if pkt.haslayer(TCP) and pkt.haslayer(IP):
            rows.append({
                "time": float(pkt.time),
                "seq": pkt[TCP].seq,
                "length": len(pkt)
            })

    df = pd.DataFrame(rows)

    if df.empty:
        raise ValueError(f"Tidak ada paket TCP di {label}")

    df = df.sort_values("time").reset_index(drop=True)

    # =============================
    # THROUGHPUT (bps)
    # =============================
    duration = df["time"].max() - df["time"].min()
    throughput = (df["length"].sum() * 8) / duration

    # =============================
    # DELAY (INTER-ARRIVAL TIME)
    # =============================
    df["delay"] = df["time"].diff()
    avg_delay = df["delay"].mean()

    # =============================
    # JITTER (VARIASI DELAY)
    # =============================
    df["jitter"] = df["delay"].diff().abs()
    avg_jitter = df["jitter"].mean()

    # =============================
    # PACKET LOSS (TCP SEQ JUMP)
    # =============================
    df["seq_diff"] = df["seq"].diff()
    threshold = df["seq_diff"].median() * 2
    packet_loss_pct = len(df[df["seq_diff"] > threshold]) / len(df) * 100

    return {
        "scenario": label,
        "duration_s": duration,
        "throughput_bps": throughput,
        "avg_delay_s": avg_delay,
        "avg_jitter_s": avg_jitter,
        "packet_loss_pct": packet_loss_pct
    }
    

In [4]:
# =============================
# ANALISIS SEBELUM & SESUDAH IPS
# =============================
res_no_ips = analyze_qos(PCAP_NO_IPS, "No IPS")
res_ips    = analyze_qos(PCAP_IPS, "With IPS")

df_result = pd.DataFrame([res_no_ips, res_ips])
print("\n=== HASIL QoS ===")
print(df_result)


=== HASIL QoS ===
   scenario  duration_s  throughput_bps  avg_delay_s  avg_jitter_s  \
0    No IPS   84.713881    1.320351e+06     0.004113      0.007260   
1  With IPS   40.789019    2.730803e+06     0.002004      0.003487   

   packet_loss_pct  
0        48.932039  
1        47.745137  


In [5]:
# =============================
# PERBANDINGAN QoS
# =============================
comparison = pd.DataFrame({
    "metric": [
        "throughput_bps",
        "avg_delay_s",
        "avg_jitter_s",
        "packet_loss_pct"
    ],
    "no_ips": [
        res_no_ips["throughput_bps"],
        res_no_ips["avg_delay_s"],
        res_no_ips["avg_jitter_s"],
        res_no_ips["packet_loss_pct"]
    ],
    "with_ips": [
        res_ips["throughput_bps"],
        res_ips["avg_delay_s"],
        res_ips["avg_jitter_s"],
        res_ips["packet_loss_pct"]
    ]
})

comparison["delta"] = comparison["with_ips"] - comparison["no_ips"]
comparison["delta_percent"] = (comparison["delta"] / comparison["no_ips"]) * 100

print("\n=== PERBANDINGAN QoS ===")
print(comparison)


=== PERBANDINGAN QoS ===
            metric        no_ips      with_ips         delta  delta_percent
0   throughput_bps  1.320351e+06  2.730803e+06  1.410452e+06     106.824053
1      avg_delay_s  4.112524e-03  2.003882e-03 -2.108642e-03     -51.273669
2     avg_jitter_s  7.260131e-03  3.486799e-03 -3.773332e-03     -51.973330
3  packet_loss_pct  4.893204e+01  4.774514e+01 -1.186902e+00      -2.425614


In [6]:
# =============================
# SIMPAN KE CSV
# =============================
output_path = "qos_goldeneye.csv"
comparison.to_csv(output_path, index=False)

print(f"Hasil perbandingan QoS disimpan ke: {output_path}")


Hasil perbandingan QoS disimpan ke: qos_goldeneye.csv
